In [31]:
from collections import Counter
from numpy import NaN, unicode_
import pandas as pd

import requests
import json 
import base64

# TODO: move these into a file/environment variable external to the code
client_ID = "27d9f488070742d2aef19da110262941"
client_secret = "e25970d2f33740a6a49bb06866abf20b"

auth_endpoint = "https://accounts.spotify.com/api/token"
search_API_endpoint = "https://api.spotify.com/v1/search"

# get access token to use for authentication with search api
def get_access_token():
    # from Spotify docs:
    # Required: Base 64 encoded string that contains the client ID and client secret key. 
    # The field must have the format: 
    # Authorization: Basic *<base64 encoded client_id:client_secret>*
    message = client_ID + ":" + client_secret
    message_bytes = message.encode("ascii")
    base64_bytes = base64.b64encode(message_bytes)
    encoded_client_details = base64_bytes.decode("ascii")
    
    headers = {"Authorization": "Basic " + encoded_client_details}              
    body = {"grant_type": "client_credentials"}
    response = requests.post(url=auth_endpoint, headers=headers, data=body)
    json_object = json.loads(response.text)
    return json_object["access_token"]
    
# make the request using requests module
# need to send the access token via request headers
def make_request(access_token, full_url):
    headers = {"Accept": "application/json", 
               "Content-Type": "application/json", 
               "Authorization": "Bearer " + access_token}

    response = requests.get(url=full_url, headers=headers)
    json_object = json.loads(response.text)

    return json_object

# create request url, make request, return JSON response
def search_request(access_token, search_term, search_type):
    search_term = requests.utils.quote(search_term)
    search_type = requests.utils.quote(search_type)
    url = search_API_endpoint + "?q=" + search_term
    url += "&type=" + search_type
    json_obj = make_request(access_token, url)
    return json_obj

# parse list of genres from JSON response
def get_genres(json_obj):
    artists = json_obj["artists"]
    items = artists["items"]
    first_artist_item = items[0] # TODO: are they sorted by match confidence/popularity?
    genres = first_artist_item["genres"]
    return genres

def run_for_art(art_name):
    access_token = get_access_token()
    json_obj = search_request(access_token, art_name, "artist")
    genres = get_genres(json_obj)
    return genres

# creating the data frame
# df0 = pd.read_json("HSStreamingHistory0.json")
# df1 = pd.read_json("HSStreamingHistory1.json")
df2 = pd.read_json("HSStreamingHistory2.json")
df3 = pd.read_json("HSStreamingHistory3.json")


df = pd.concat([df3])

df["endHour"] = df["endTime"].str.split(' ').str[1].str.split(':').str[0].astype(int)
df["date"] = df["endTime"].str.split(' ').str[0]
df = df.drop("endTime", axis=1)
days = pd.read_csv("DaysofWeek.csv")
df = df.merge(days, on=["date"], how="inner")

group_by_day = df.groupby("days of week")
fri_df = group_by_day.get_group("Friday")
sat_df = group_by_day.get_group("Saturday")
sun_df = group_by_day.get_group("Sunday")
mon_df = group_by_day.get_group("Monday")
tue_df = group_by_day.get_group("Tuesday")
wed_df = group_by_day.get_group("Wednesday")
thur_df = group_by_day.get_group("Thursday")

weekday_df = mon_df

weekday_df = weekday_df.append(tue_df)
weekday_df = weekday_df.append(wed_df)
weekday_df = weekday_df.append(thur_df)
weekday_df = weekday_df.append(fri_df)

## first group by date
dates = weekday_df.groupby("date")


HSdf_date = pd.DataFrame()

HSdf_date["date"] = dates.groups.keys()
# HSdf_date["day of week"] = df["days of week"]

mode_art = []
mode_genere = []

for date in HSdf_date["date"]:
    item = dates.get_group(date)["artistName"].mode()[0]
    mode_art.append(item)

HSdf_date["mode artist"] = mode_art

for artist in HSdf_date["mode artist"]:
    if len(run_for_art(artist)) > 0:
        genere = run_for_art(artist)[0]
        mode_genere.append(genere)
    else:
        mode_genere.append(NaN)

HSdf_date["mode genre"] = mode_genere

HSdf_date.drop([6, 47], inplace=True)


LWdf1 = pd.read_json("LWStreamingHistory1.json")
LWdf2 = pd.read_json("LWStreamingHistory2.json")


df = pd.concat([LWdf1, LWdf2])

df["endHour"] = df["endTime"].str.split(' ').str[1].str.split(':').str[0].astype(int)
df["date"] = df["endTime"].str.split(' ').str[0]
df = df.drop("endTime", axis=1)
days = pd.read_csv("DaysofWeek.csv")
df = df.merge(days, on=["date"], how="inner")
df.dropna()

group_by_day = df.groupby("days of week")
fri_df = group_by_day.get_group("Friday")
sat_df = group_by_day.get_group("Saturday")
sun_df = group_by_day.get_group("Sunday")
mon_df = group_by_day.get_group("Monday")
tue_df = group_by_day.get_group("Tuesday")
wed_df = group_by_day.get_group("Wednesday")
thur_df = group_by_day.get_group("Thursday")

weekday_df = mon_df

weekday_df = weekday_df.append(tue_df)
weekday_df = weekday_df.append(wed_df)
weekday_df = weekday_df.append(thur_df)
weekday_df = weekday_df.append(fri_df)

## first group by date
dates = weekday_df.groupby("date")


LWdf_date = pd.DataFrame()

LWdf_date["date"] = dates.groups.keys()
# LWdf_date["day of week"] = df["days of week"]

mode_art = []
mode_genere = []

for date in LWdf_date["date"]:
    item = dates.get_group(date)["artistName"].mode()[0]
    mode_art.append(item)

LWdf_date["mode artist"] = mode_art

for artist in LWdf_date["mode artist"]:
    if len(run_for_art(artist)) > 0:
        genere = run_for_art(artist)[0]
        mode_genere.append(genere)
    else:
        mode_genere.append(NaN)

LWdf_date["mode genre"] = mode_genere


In [33]:
import matplotlib.pyplot as plt
import numpy as np
print(HSdf_date)



X = HSdf_date["mode genre"]
Y = close_price


from scipy import stats

stocks = pd.read_csv("stocks.csv")
close_price = stocks["Close"]

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X = labelencoder.fit_transform(X)
print(X)



unseen_x = 0
X_train = np.reshape(X, (-1, 1))
y_train = Y
X_test = [[0]] # Afrofuturism
# print(X)



# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_predicted = lin_reg.predict(X_test)
print("y predicted:", y_predicted)




          date            mode artist           mode genre
0   2021-12-15         Franz Schubert            classical
1   2021-12-16         Franz Schubert            classical
2   2021-12-17         Franz Schubert            classical
3   2021-12-21     Neutral Milk Hotel     alternative rock
4   2021-12-22            Beach House      baltimore indie
5   2021-12-23             Mac Miller              hip hop
7   2021-12-27          Billy Strings             jam band
8   2021-12-28          Billy Strings             jam band
9   2021-12-29        Indigo De Souza              art pop
10  2021-12-30        Indigo De Souza              art pop
11  2021-12-31        Indigo De Souza              art pop
12  2022-01-03             Thundercat         afrofuturism
13  2022-01-04            Beach House      baltimore indie
14  2022-01-05            James Blake      alternative r&b
15  2022-01-06          L'Impératrice     french indie pop
16  2022-01-07           Jason Isbell  alternative count